<H2> Assignment 25 Xgboost </H2>

<H5> Importing Libraries </H5>

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

<H3> Load Train and test data  sets </H3>

In [2]:
colnames=['age','workclass','fnlwgt','education','educationNum','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']

In [3]:
train_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',names=colnames, header = None)

In [4]:
test_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test', skiprows = 1, names = colnames, header = None)

In [5]:
train_set.shape

(32561, 15)

In [6]:
test_set.shape

(16281, 15)

In [7]:
train_set.head()

,age,workclass,fnlwgt,education,educationNum,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [8]:
test_set.head()

,age,workclass,fnlwgt,education,educationNum,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


<H3> Check if there are any missing values</H3>

In [9]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
educationNum      32561 non-null int64
marital-status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital-gain      32561 non-null int64
capital-loss      32561 non-null int64
hours-per-week    32561 non-null int64
native-country    32561 non-null object
income            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [10]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16281 entries, 0 to 16280
Data columns (total 15 columns):
age               16281 non-null int64
workclass         16281 non-null object
fnlwgt            16281 non-null int64
education         16281 non-null object
educationNum      16281 non-null int64
marital-status    16281 non-null object
occupation        16281 non-null object
relationship      16281 non-null object
race              16281 non-null object
sex               16281 non-null object
capital-gain      16281 non-null int64
capital-loss      16281 non-null int64
hours-per-week    16281 non-null int64
native-country    16281 non-null object
income            16281 non-null object
dtypes: int64(6), object(9)
memory usage: 1.9+ MB


<H3> We need to removed few unknowns which are labeled as ? </H3> 

In [11]:
train_set.replace(' ?', np.nan).dropna().shape

(30162, 15)

In [12]:
test_set.replace(' ?', np.nan).dropna().shape

(15060, 15)

In [13]:
train_nomissing = train_set.replace(' ?', np.nan).dropna()
test_nomissing = test_set.replace(' ?', np.nan).dropna()
#train_set = train_set.replace(' ?', np.nan).dropna()
#test_set = test_set.replace(' ?', np.nan).dropna()


In [14]:
# we are dropping the periods so both test and train income unique value matches
test_nomissing['income'] = test_nomissing.income.replace({' <=50K.': ' <=50K', ' >50K.':' >50K'})
test_nomissing.income.unique()
#train_set['income'] = train_set.income.replace({' <=50K.': ' <=50K', ' >50K.':' >50K'})
#train_set.income.unique()


array([' <=50K', ' >50K'], dtype=object)

In [15]:
#train_nomissing.income.unique()
train_nomissing.income.unique()

array([' <=50K', ' >50K'], dtype=object)

In [16]:
# we should combine both the datasets into one single one
combined_set = pd.concat([train_nomissing, test_nomissing], axis = 0) # Stacks them vertically


In [17]:
combined_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 16280
Data columns (total 15 columns):
age               45222 non-null int64
workclass         45222 non-null object
fnlwgt            45222 non-null int64
education         45222 non-null object
educationNum      45222 non-null int64
marital-status    45222 non-null object
occupation        45222 non-null object
relationship      45222 non-null object
race              45222 non-null object
sex               45222 non-null object
capital-gain      45222 non-null int64
capital-loss      45222 non-null int64
hours-per-week    45222 non-null int64
native-country    45222 non-null object
income            45222 non-null object
dtypes: int64(6), object(9)
memory usage: 5.5+ MB


In [18]:
# if the feature is not already numerical, we need to encode it as one
for feature in combined_set.columns: # Loop through all columns in the dataframe
    if combined_set[feature].dtype == 'object': # Only apply for columns with categorical strings
        combined_set[feature] = pd.Categorical(combined_set[feature]).codes # Replace strings with an integer


In [88]:
combined_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 16280
Data columns (total 15 columns):
age               45222 non-null int64
workclass         45222 non-null int8
fnlwgt            45222 non-null int64
education         45222 non-null int8
educationNum      45222 non-null int64
marital-status    45222 non-null int8
occupation        45222 non-null int8
relationship      45222 non-null int8
race              45222 non-null int8
sex               45222 non-null int8
capital-gain      45222 non-null int64
capital-loss      45222 non-null int64
hours-per-week    45222 non-null int64
native-country    45222 non-null int8
income            45222 non-null int8
dtypes: int64(6), int8(9)
memory usage: 2.8 MB


In [19]:
# Split them back to train and test datasets
final_train = combined_set[:train_nomissing.shape[0]] # Up to the last initial training set row
final_test = combined_set[train_nomissing.shape[0]:] # Past the last initial training set row


In [20]:
final_train.shape

(30162, 15)

In [21]:
final_test.shape

(15060, 15)

In [22]:
# Assigning the target value to y train and y test 
y_train = final_train.pop('income')
y_test = final_test.pop('income')

In [23]:
xgdmat = xgb.DMatrix(final_train, y_train)

In [24]:
#Params settings
params = {
    'objective':'binary:logistic',
    'max_depth':2,
    'silent':1,
    'eta':1
}
num_rounds = 5

In [26]:
cv_xgb = xgb.cv(params = params, dtrain = xgdmat, num_boost_round = 5, nfold = 5)

In [27]:
final_gb = xgb.train(params, xgdmat, num_rounds)

In [29]:
testdmat = xgb.DMatrix(final_test)

In [30]:
from sklearn.metrics import accuracy_score
y_pred = final_gb.predict(testdmat) # Predict using our testdmat
y_pred


array([0.00953897, 0.36890996, 0.09991371, ..., 0.83994377, 0.31412885,
       0.76152873], dtype=float32)

In [31]:
# with threshold as 0.5 we will convert the output to 0 and 1
y_pred[y_pred > 0.5] = 1
y_pred[y_pred <= 0.5] = 0
y_pred

array([0., 0., 0., ..., 1., 0., 1.], dtype=float32)

In [32]:
# Accuracy score
accuracy_score(y_pred, y_test), 1-accuracy_score(y_pred, y_test)


(0.8433598937583001, 0.15664010624169988)

<H3>Our final accuracy is 84.33%, or a 15.66% error rate </H3>